### Packages

In [1]:
from prodigy.components.db import connect
import spacy
import pandas as pd

## Personal Solution

<p> Extract Tokens and Tags in list (change "O-" with "O"), append each list into a column of Dataframe, combine columns and export them to a txt.file </p>

In [28]:
words =[]

db = connect()
prodigy_annotations = db.get_dataset("orgs_pd")
examples = ((eg["text"], eg) for eg in prodigy_annotations)
nlp =spacy.load("el_core_news_sm")
for doc, eg in nlp.pipe(examples, as_tuples=True):
    for token in eg["tokens"]:
        words.append(token["text"])
    words.append("\n") #insert gap required for IOB tagging
len(words)


16853

In [29]:
tags=[]
prodigy_annotations = db.get_dataset("orgs_pd")
examples = ((eg["text"], eg) for eg in prodigy_annotations)
nlp =spacy.load("el_core_news_sm")
for doc, eg in nlp.pipe(examples, as_tuples=True):
    doc.ents = [doc.char_span(s["start"], s["end"], s["label"]) for s in eg["spans"]]
    for t in doc:
        if t.ent_iob_:
            tags.append(f"{t.ent_iob_}-{t.ent_type_}")
        else:
            tags.append("O")
    tags.append("\n")  #insert gap required for IOB tagging
tags = [w.replace('O-', 'O') for w in tags]
len(tags)

16853

In [30]:
df = pd.DataFrame(
    {'Tokens': words,
    'Tags': tags,
     })

In [31]:
df[0:15]

,Tokens,Tags
0,"""",O
1,1.,O
2,Αποστολή,O
3,του,O
4,Υπουργείου,B-ORG
5,Ανάπτυξης,I-ORG
6,και,I-ORG
7,Επενδύσεων,I-ORG
8,είναι,O
9,η,O


In [32]:
df['join'] = df['Tokens'] + ' ' + df['Tags']
df['join'][0:15]

0                  " O
1                 1. O
2           Αποστολή O
3                του O
4     Υπουργείου B-ORG
5      Ανάπτυξης I-ORG
6            και I-ORG
7     Επενδύσεων I-ORG
8              είναι O
9                  η O
10          ισόρροπη O
11        οικονομική O
12          ανάπτυξη O
13               των O
14       περιφερειών O
Name: join, dtype: object

In [33]:
with open('/Users/alexmichailidis/gg-extraction-2022/data/ORG_IOB_tags.txt', 'w') as f:
    for text in df['join'].tolist():
        f.write(text + '\n')

## Found Solutuion (Partial)

In [ ]:
#convert to IOB format
from prodigy.components.db import connect
import spacy

db = connect()
prodigy_annotations = db.get_dataset("pd_ner")
examples = ((eg["text"], eg) for eg in prodigy_annotations)
nlp =spacy.load("el_core_news_sm")
for doc, eg in nlp.pipe(examples, as_tuples=True):
    doc.ents = [doc.char_span(s["start"], s["end"], s["label"]) for s in eg["spans"]]
    iob_tags = [f"{t.ent_iob_}-{t.ent_type_}" if t.ent_iob_ else "O" for t in doc]
    print(doc.text, iob_tags)

## Found Solutuion (Token Based Converted)

In [ ]:
db = connect()
prodigy_annotations = db.get_dataset("pd_ner")
examples = ((eg["text"], eg) for eg in prodigy_annotations)
nlp =spacy.load("el_core_news_sm")
for doc, eg in nlp.pipe(examples, as_tuples=True):
    words = [token["text"] for token in eg["tokens"]]
    doc.ents = [doc.char_span(s["start"], s["end"], s["label"]) for s in eg["spans"]]
    iob_tags = [f"{t.ent_iob_}-{t.ent_type_}" if t.ent_iob_ else "O" for t in doc]
    print(words, iob_tags)